In [1]:
from dataloader import AtariDataset
import gym
import torch.nn as nn
import torch
import numpy as np
import random
import tqdm
from tqdm import tqdm
import torch.nn.functional as F
from torch.optim import optimizer
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

## SEEDING

In [2]:
def reseed(seed):
  torch.manual_seed(seed)
  random.seed(seed)
  np.random.seed(seed)
seed = 42
reseed(seed)

## LOAD DATA

In [3]:
dataloader = AtariDataset("atari_v1")
observations, actions, rewards, next_observations, dones = dataloader.compile_data()

5
[1960, 1870, 1770, 1705, 1700]


ValueError: too many values to unpack (expected 2)

## MAKE ENVIRONMENT

In [ ]:
def make_env(env_id, seed=25):
    env = gym.make(env_id, obs_type='grayscale', render_mode=None, repeat_action_probability=0.15,frameskip=1)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env
env = make_env("SpaceInvaders-v0", seed=seed)
print(env.action_space.n)
print(env.observation_space.shape)




6
(210, 160)


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


# TRAIN DQN (TEST)

In [ ]:

from dqn import DQN
import dqn

INPUT_SHAPE = (210, 160)
ACTION_SIZE = env.action_space.n

dqn_learner = DQN(INPUT_SHAPE, ACTION_SIZE)

dqn.train(dqn_learner, env, observations=observations, actions=actions, rewards=rewards, next_observations=next_observations, dones=dones, save_path='models/dqn_test.pth')

## Train BC

In [ ]:
from bc import SpaceInvLearner
import bc

learner = SpaceInvLearner(env)

bc.train(learner=learner, observations=observations, checkpoint_path="models/bc_learner.pth", actions=actions, num_epochs=40)

Training the learner
Training for 40 epochs


  2%|▎         | 1/40 [00:19<12:49, 19.73s/it]

Epoch 0, Loss: 0.2328628909548205


  5%|▌         | 2/40 [00:39<12:27, 19.67s/it]

Epoch 1, Loss: 0.09765370158450021


...

 98%|█████████▊| 39/40 [11:10<00:17, 17.33s/it]

Epoch 38, Loss: 0.030496925144869083


100%|██████████| 40/40 [11:27<00:00, 17.20s/it]

Epoch 39, Loss: 0.029993618590682898


SpaceInvLearner(
  (fc1): Linear(in_features=33600, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=6, bias=True)
)

In [ ]:
learner.load_state_dict(torch.load("models/bc_learner.pth"), strict=True)
total_learner_reward = 0
done = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for i in range(20):
    obs = env.reset()
    done = False
    while not done:
        with torch.no_grad():
            action = learner.get_action(torch.Tensor([obs]).to(device))
        obs, reward, done, info = env.step(action)
        total_learner_reward += reward
        if done:
            break

print(total_learner_reward/20)

127.25


## LOAD EXPERT

In [ ]:
from expert.ppo import PPOAgent, ActorCnn, CriticCnn

INPUT_SHAPE = (4, 84, 84)
ACTION_SIZE = env.action_space.n
SEED = 0
GAMMA = 0.99           # discount factor
ALPHA= 0.0001          # Actor learning rate
BETA = 0.0001          # Critic learning rate
TAU = 0.95
BATCH_SIZE = 32
PPO_EPOCH = 5
CLIP_PARAM = 0.2
UPDATE_EVERY = 1000     # how often to update the network 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
agent = PPOAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, TAU, UPDATE_EVERY, BATCH_SIZE, PPO_EPOCH, CLIP_PARAM, ActorCnn(INPUT_SHAPE, ACTION_SIZE), CriticCnn(INPUT_SHAPE))
agent.load_model("models/expert_actor.pth", device)

## DAgger Implementation

In [ ]:
import dagger

dagger.interact(env, learner, agent, observations=observations, actions=actions, checkpoint_path="models/DAgger.pth", seed=seed, num_epochs=40, tqdm_disable=True)

After interaction 0, reward = 50.0
Training the learner
Training for 20 epochs
Epoch 0, Loss: 0.2291255757028674
Epoch 1, Loss: 0.17323820734465564
Epoch 2, Loss: 0.1555330808515902
Epoch 3, Loss: 0.14674009106777333
Epoch 4, Loss: 0.14217397589006542
Epoch 5, Loss: 0.13803996384879688
Epoch 6, Loss: 0.1353701772513213
Epoch 7, Loss: 0.1326952221584909
Epoch 8, Loss: 0.1309829733253997
Epoch 9, Loss: 0.12841376533478865
Epoch 10, Loss: 0.1281951579414768
Epoch 11, Loss: 0.12567432427111966
Epoch 12, Loss: 0.12410514976507352
Epoch 13, Loss: 0.12316190092651932
Epoch 14, Loss: 0.1219550437397427
Epoch 15, Loss: 0.12118014113770591
Epoch 16, Loss: 0.1200339741728924
Epoch 17, Loss: 0.11922966596888906
Epoch 18, Loss: 0.11753706652441143
Epoch 19, Loss: 0.11707328039186972
After interaction 1, reward = 135.0
Training the learner
Training for 20 epochs
Epoch 0, Loss: 0.1677919045420297
Epoch 1, Loss: 0.1543796840376021
Epoch 2, Loss: 0.1446054458709094
Epoch 3, Loss: 0.14156718197485402
Ep

In [ ]:
learner.load_state_dict(torch.load("models/DAgger.pth"), strict=True)
total_learner_reward = 0
done = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for i in range(20):
    obs = env.reset()
    done = False
    while not done:
        with torch.no_grad():
            action = learner.get_action(torch.Tensor([obs]).to(device))
        obs, reward, done, info = env.step(action)
        total_learner_reward += reward

print(total_learner_reward/20)

118.25


In [ ]:
from dqn import DQN
import dqn

INPUT_SHAPE = (210, 160)
ACTION_SIZE = env.action_space.n

dqn_learner = DQN(INPUT_SHAPE, ACTION_SIZE)

dqn.train(dqn_learner, env, )

TypeError: train() missing 6 required positional arguments: 'observations', 'actions', 'rewards', 'next_observations', 'dones', and 'save_path'